# Proof of concept example (ECAS-B2SHARE)

## 1. Use Case 

![](https://github.com/SofianeB/ECAS-B2SHARE/raw/master/demo-ecas-ophidia/ecas-b2share.png)

### 2. Calculate number of tropical nights with Ophidia

In [ ]:
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

In [ ]:
mintemp = cube.Cube(src_path='/data-training/E63_kp40_198001_199012_T2m_daymin_merged.nc',
    measure='temp2',
    import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=4,
    description='Min Temps'
    )

In [ ]:
tropicalnights = mintemp.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-293.15','>0','1','0')"
)

In [ ]:
count = tropicalnights.reduce2(
    operation='sum',
    dim='time',
    concept_level='y',
)

Export result to netCDF 

In [ ]:
count.exportnc2(output_path='/data/ecas/b2share', output_name='', export_metadata='no')

In [ ]:
firstyear = count.subset(subset_filter=1, subset_dims='time')

Plot the result 

In [ ]:
%matplotlib inline
data = firstyear.export_array(show_time='yes')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
import numpy as np

lat = data['dimension'][0]['values'][:]
lon = data['dimension'][1]['values'][:]
var = data['measure'][0]['values'][:]
var = np.reshape(var, (len(lat), len(lon)))

fig = plt.figure(figsize=(15, 15), dpi=100)
ax  = fig.add_axes([0.1,0.1,0.8,0.8])

map = Basemap(projection='cyl',llcrnrlat= -90,urcrnrlat= 90, llcrnrlon=0,urcrnrlon=360, resolution='c')

map.drawcoastlines()
map.drawparallels(np.arange( -90, 90,30),labels=[1,0,0,0])
map.drawmeridians(np.arange(-180,180,30),labels=[0,0,0,1])

x, y = map(*np.meshgrid(lon,lat))

clevs = np.arange(0,371,10)

cnplot = map.contourf(x,y,var,clevs,cmap=plt.cm.jet)
cbar = map.colorbar(cnplot,location='right')

plt.title('Tropical Nights (year 1980)')
plt.savefig('tn.png')
plt.show()

### 3. Create draft record in B2SHARE and add file(s) 

#### Install ecasb2share Python library

In [ ]:
# !pip install ecasb2share

In [ ]:
from ecasb2share.ecasb2shareclient import EcasShare as Client
client = Client(url='https://eudat-b2share-test.csc.fi', token_file='/home/jovyan/work/token.txt')

In [ ]:
client.list_communities()

In [ ]:
title = 'Tropical Nights 30. August'
file = 'b2share/temp2_4595.nc'
figure = 'tn.png'
filebucketid = '1b76baae-6cf9-4a09-8a76-7a312f80f60a'

In [ ]:
client.create_draft_record_with_pid(title, 'pid1')

####  Add exported dataset (.nc) to the draft record 

In [ ]:
client.add_file_to_draft_record(file, filebucketid)

#### Add plot figure to the draft record

In [ ]:
client.add_file_to_draft_record(figure, filebucketid)

## 4. Delete draft record and clean workspace

In [ ]:
draft_record_id = ''
client.delete_draft_record(draft_record_id)

In [ ]:
cube.Cube.client.submit("oph_delete cube=[container=E63_kp40_198001_199012_T2m_daymin_merged.nc]")
cube.Cube.deletecontainer(container='E63_kp40_198001_199012_T2m_daymin_merged.nc')